

---

## Importing Relavant Libraries  and accessing dataset from google drive via linkn of dataset

In [ ]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
# df_train = pd.read_csv(r"phighing dataset.csv")
df_train = pd.read_csv(r"/content/drive/MyDrive/phighing dataset.csv")

df=df_train.copy()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df

In [ ]:
df.to_csv("phishing.csv")

**Analysis of Data **

In [ ]:
df.isnull().sum().sum()


0

In [ ]:
statistics_of_data = []
for col in df.columns:
  statistics_of_data.append((col,
                             df[col].nunique(),
                             df[col].isnull().sum()*100/df.shape[0],
                             df[col].value_counts(normalize=True, dropna=False).values[0] * 100, 
                             df[col].dtype
                             ))
stats_df = pd.DataFrame(statistics_of_data, columns=['Feature', 'Uniq_val', 'missing_val', 'val_biggest_cat', 'type'])

In [ ]:
stats_df

,Feature,Uniq_val,missing_val,val_biggest_cat,type
0,qty_dot_url,23,0.0,50.498764,int64
1,qty_hyphen_url,30,0.0,76.809617,int64
2,qty_underline_url,22,0.0,90.983034,int64
3,qty_slash_url,25,0.0,29.852502,int64
4,qty_questionmark_url,6,0.0,98.722824,int64
...,...,...,...,...,...
107,qty_redirects,15,0.0,60.286469,int64
108,url_google_index,3,0.0,99.858470,int64
109,domain_google_index,3,0.0,99.769801,int64
110,url_shortened,2,0.0,99.171285,int64


In [ ]:
df[df.duplicated(keep='first')]

,qty_dot_url,qty_hyphen_url,qty_underline_url,qty_slash_url,qty_questionmark_url,qty_equal_url,qty_at_url,qty_and_url,qty_exclamation_url,qty_space_url,...,qty_ip_resolved,qty_nameservers,qty_mx_servers,ttl_hostname,tls_ssl_certificate,qty_redirects,url_google_index,domain_google_index,url_shortened,phishing
168,1,0,0,0,0,0,0,0,0,0,...,-1,4,0,-1,0,-1,0,0,0,0
321,1,0,0,0,0,0,0,0,0,0,...,-1,4,0,-1,0,-1,0,0,0,0
1131,1,0,0,0,0,0,0,0,0,0,...,-1,4,0,-1,0,-1,0,0,0,0
1445,2,0,0,0,0,0,0,0,0,0,...,-1,4,0,-1,0,-1,0,0,0,0
1901,2,0,0,0,0,0,0,0,0,0,...,-1,2,1,-1,0,-1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58366,2,0,0,0,0,0,0,0,0,0,...,-1,4,0,-1,0,-1,0,0,0,0
58424,2,0,0,0,0,0,0,0,0,0,...,-1,4,0,-1,0,-1,0,0,0,0
58518,1,0,0,0,0,0,0,0,0,0,...,-1,4,0,-1,0,-1,0,0,0,0
58533,1,0,0,0,0,0,0,0,0,0,...,-1,4,0,-1,0,-1,0,0,0,0


In [ ]:
df.drop_duplicates(keep = 'first', inplace = True)
df.shape

(57405, 112)

In [ ]:
y=df.iloc[:, 111].values
x = df.drop('phishing',axis=1).values
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state = 42, test_size = 0.2)

In [ ]:
from sklearn.metrics import accuracy_score

## Simple RF

In [ ]:

rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
rfc.score(x_train, y_train)
y_predict = rfc.predict(x_test)

In [ ]:
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(df)

VarianceThreshold(threshold=0)

In [ ]:
data =df.copy()
constant_columns = [column for column in data.columns
                    if column not in data.columns[var_thres.get_support()]]

print(len(constant_columns))

13


In [ ]:
data.drop(constant_columns,axis=1,inplace=True)

In [ ]:
y=data.loc[:,'phishing']
x = data.drop('phishing',axis=1) 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state = 42, test_size = 0.1)


In [ ]:
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
rfc.score(x_train, y_train)
y_predict = rfc.predict(x_test)

In [ ]:
round(accuracy_score(y_test, y_predict),3)*100

95.5

In [ ]:
def correlation(dataset, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname] # deleting the column from the dataset

    return col_corr

In [ ]:
corr_features = correlation(x_train, 0.7)
len(corr_features)
# print(d1)
# print(d2)
# print(pair_list)

49

In [ ]:
x_test.drop(corr_features,axis=1,inplace=True)

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
rfc.score(x_train, y_train)
y_predict = rfc.predict(x_test)

In [ ]:
round(accuracy_score(y_test, y_predict),3)*100

95.8

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [ ]:
import numpy as np
n_estimators = list(range(100,2000,200)) # number of trees in the random forest
criterion=['entropy', 'gini']
max_features = ['auto', 'sqrt','log2'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree

min_samples_split = [2, 4,6,8, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4,7] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,

'max_features': max_features,
'criterion':criterion,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap}


In [ ]:
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
clf = RandomForestClassifier()
acc_scorer = make_scorer(accuracy_score)

# Run the grid search
grid_obj = GridSearchCV(clf, random_grid, scoring=acc_scorer)
grid_obj = grid_obj.fit(x_train, y_train)

# Set the clf to the best combination of parameters
clf = grid_obj.best_estimator_

# Fit the best algorithm to the data. 
clf.fit(x_train, y_train)